In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import scipy
from skimage import color
import tempfile
import os
from timeit import default_timer as timer
from tempfile import TemporaryFile
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process.kernels import WhiteKernel
from sklearn.gaussian_process.kernels import ExpSineSquared
import decimal
from decimal import *
from sklearn.gaussian_process.kernels import ConstantKernel
from numpy import loadtxt
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import keras
from sklearn.metrics import roc_auc_score
import pickle


In [2]:
import h5py
import matplotlib as mpl

In [3]:
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

In [4]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
def Confus_analyzer(class_array, pred_array):
    
    TN, FP, FN, TP = confusion_matrix(class_array, val_pred).ravel()
    
    Accuracy = ((TP+TN)/(TP+TN+FP+FN))
    print('Accuracy: ' + str(Accuracy))
    #Sensitivity or Recall; rate of TP on all positives
    Recall = ((TP)/(TP+FN))
    print('Recall: ' + str(Recall))
    Misclass = ((FP+FN)/(TP+TN+FP+FN))
    print('Misclassifications: ' + str(Misclass))
    #Percentage of correct TP predictions
    Precision = ((TP)/(TP+FP))
    print('Precision: ' + str(Precision))
    #F1 score
    F1_score = 2*((Precision*Recall)/(Precision+Recall))
    print('F1_Score' + str(F1_score))
    ROC_AUC = roc_auc_score(class_array, pred_array)
    print('ROC_AUC: ' + str(ROC_AUC))
    
    
    return()

In [6]:
def Dataframe_processor(DF_path_1, DF_path_2, appendmode=False):
    
    if appendmode == True:
    
        DF_1 = pickle.load(open(DF_path_1, 'rb'))
        DF_2 = pickle.load(open(DF_path_2, 'rb'))


        DF_C = DF_1.append(DF_2)
        DF_C = DF_C.reset_index(drop=True)


        DF_C_signals = DF_C.iloc[:, 0:(DF_C.shape[1]-1)]
        DF_C_signals = DF_C_signals.to_numpy(copy=True)


        DF_C_classes = DF_C.iloc[:, (DF_C.shape[1]-1):DF_C.shape[1]]
        DF_C_classes = DF_C_classes.to_numpy(copy=True)
    
        return(DF_C_signals, DF_C_classes)
    
    if appendmode == False:
        
        DF_C = pickle.load(open(DF_path_1, 'rb'))
        
        DF_C_signals = DF_C.iloc[:, 0:(DF_C.shape[1]-1)]
        DF_C_signals = DF_C_signals.to_numpy(copy=True)


        DF_C_classes = DF_C.iloc[:, (DF_C.shape[1]-1):DF_C.shape[1]]
        DF_C_classes = DF_C_classes.to_numpy(copy=True)
    
        return(DF_C_signals, DF_C_classes)
    

In [7]:
#pickle_off = open("Emp.pickle","rb")
#emp = pickle.load(pickle_off)
#print(emp)

In [8]:
train_df_22 ='/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Split_dfs/P22/P22_train_SMALL.pkl'
train_df_109C ='/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Split_dfs/P109C/P109_train_SMALL.pkl'
val_df_22 = '/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Split_dfs/P22/P22_val_SMALL.pkl'
val_df_109 = '/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Split_dfs/P109C/P109_val_SMALL.pkl'
test_df_22 = '/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Split_dfs/P22/P22_test_SMALL.pkl'
test_df_109 = '/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Split_dfs/P109C/P109_test_SMALL.pkl'

In [9]:
#tr_sig_109, tr_cl_109 = Dataframe_processor(train_df_109C, None, appendmode=False)

In [10]:
#tr_sig_109 = np.nan_to_num(tr_sig_109)

In [11]:
#tr_sig_22, tr_cl_22 = Dataframe_processor(train_df_22, None, appendmode=False)

In [12]:
train_signals, train_classes = Dataframe_processor(train_df_22, train_df_109C, appendmode=True)

In [13]:
val_signals, val_classes = Dataframe_processor(val_df_22, val_df_109, appendmode=True)

In [14]:
test_signals, test_classes = Dataframe_processor(test_df_22, test_df_109, appendmode=True)

In [15]:
train_signals = np.nan_to_num(train_signals)
val_signals = np.nan_to_num(val_signals)
test_signals = np.nan_to_num(test_signals)


In [16]:
uniques, occurrences = np.unique(train_classes, return_counts=True)
ratio = occurrences[1]/(occurrences[0]+occurrences[1])

In [17]:
class_weight_1 = 1/ratio

In [18]:
class_weight_1

235.41792666099056

In [19]:
Metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='AUC'),
    keras.metrics.AUC(name='prc', curve='PR'),
]



In [20]:
def Modelbuilder(train_signals,
                 metrics=Metrics,
                 learning_rate=1e-4):

    
    model = keras.Sequential([
        keras.layers.Dense(
            512, activation='relu',
            input_shape=(train_signals.shape[1],)),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')])
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate=1e-4),
        loss = keras.losses.BinaryCrossentropy(),
        metrics=metrics)
    
    return(model)

In [21]:
def Modelbuilder_ada(train_signals,
                 metrics=Metrics,
                 learning_rate=1e-4):

    
    model = keras.Sequential([
        keras.layers.Dense(
            512, activation='relu',
            input_shape=(train_signals.shape[1],)),
        keras.layers.Dense(256, activation='relu'),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')])
    model.compile(
        optimizer = keras.optimizers.Adagrad(learning_rate=1e-4),
        loss = keras.losses.BinaryCrossentropy(),
        metrics=metrics)
    
    return(model)

In [22]:
model1 = Modelbuilder(train_signals, metrics=Metrics, learning_rate=1e-4)
model2 = Modelbuilder(train_signals, metrics=Metrics, learning_rate=1e-4)
model3 = Modelbuilder(train_signals, metrics=Metrics, learning_rate=1e-5)
model4 = Modelbuilder(train_signals, metrics=Metrics, learning_rate=1e-6)
model5 = Modelbuilder(train_signals, metrics=Metrics, learning_rate=1e-4)
model6 = Modelbuilder(train_signals, metrics=Metrics, learning_rate=1e-4)
model7 = Modelbuilder_ada(train_signals, metrics=Metrics, learning_rate=1e-4)
model8 = Modelbuilder_ada(train_signals, metrics=Metrics, learning_rate=1e-4)
model9 = Modelbuilder_ada(train_signals, metrics=Metrics, learning_rate=1e-4)
model10 = Modelbuilder_ada(train_signals, metrics=Metrics, learning_rate=1e-4)


In [ ]:
#Standard reference
model1_history = model1.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 128,
    class_weight={0:1, 1:class_weight_1},
    validation_data=(val_signals, val_classes))

Epoch 1/5
62646/62646 [==============================] - 358s 6ms/step - loss: 1.2573 - tp: 11957.4651 - tn: 2425139.4204 - fp: 1567232.3201 - fn: 5142.7886 - accuracy: 0.5996 - precision: 0.0073 - recall: 0.6854 - AUC: 0.6977 - prc: 0.0097 - val_loss: 0.6107 - val_tp: 6902.0000 - val_tn: 1509750.0000 - val_fp: 771604.0000 - val_fn: 2812.0000 - val_accuracy: 0.6620 - val_precision: 0.0089 - val_recall: 0.7105 - val_AUC: 0.7564 - val_prc: 0.0148
Epoch 2/5
62646/62646 [==============================] - 349s 6ms/step - loss: 1.1525 - tp: 12105.9708 - tn: 2704445.1034 - fp: 1288028.6854 - fn: 4892.2347 - accuracy: 0.6770 - precision: 0.0093 - recall: 0.7141 - AUC: 0.7627 - prc: 0.0157 - val_loss: 0.6734 - val_tp: 7568.0000 - val_tn: 1405593.0000 - val_fp: 875761.0000 - val_fn: 2146.0000 - val_accuracy: 0.6168 - val_precision: 0.0086 - val_recall: 0.7791 - val_AUC: 0.7695 - val_prc: 0.0173
Epoch 3/5
41613/62646 [==================>...........] - ETA: 1:33 - loss: 1.1208 - tp: 8010.5716 - tn

In [ ]:
#No class weights
model2_history = model2.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 128,
    validation_data=(val_signals, val_classes))

In [ ]:
#Learning rate 1e-5
model3_history = model3.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 128,
    class_weight={0:1, 1:class_weight_1},
    validation_data=(val_signals, val_classes))

In [ ]:
#Learning rate 1e-6
model4_history = model4.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 128,
    class_weight={0:1, 1:class_weight_1},
    validation_data=(val_signals, val_classes))

In [ ]:
#Batch size 512
model5_history = model5.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 512,
    class_weight={0:1, 1:class_weight_1},
    validation_data=(val_signals, val_classes))

In [ ]:
#Batch size 64
model6_history = model6.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 64,
    class_weight={0:1, 1:class_weight_1},
    validation_data=(val_signals, val_classes))

In [ ]:
#Adagrad optimizer
model7_history = model7.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 128,
    class_weight={0:1, 1:class_weight_1},
    validation_data=(val_signals, val_classes))

In [ ]:
#Adagrad optimizer, no class weights
model8_history = model8.fit(
    train_signals,
    train_classes,
    epochs = 5,
    batch_size = 128,
    validation_data=(val_signals, val_classes))

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model1_history_SMALL.npy', model1_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model2_history_SMALL.npy', model2_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model3_history_SMALL.npy', model3_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model4_history_SMALL.npy', model4_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model5_history_SMALL.npy', model5_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model6_history_SMALL.npy', model6_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model7_history_SMALL.npy', model7_history.history)

In [ ]:
np.save('/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model8_history_SMALL.npy', model8_history.history)

In [ ]:
#Example code to load np.history
#history1=np.load('history1.npy',allow_pickle='TRUE').item()


In [ ]:
NN_model.save('/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Neural_networks/NN_1_B32_adam_SMALL.h5')

In [ ]:
def Modelsaver(NN_model, path_and_name):
    
    NN_model.save(path_and_name + '.h5')

In [ ]:
Modelsaver(
    model1,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model1_SMALL')

In [ ]:
Modelsaver(
    model2,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model2_SMALL')

In [ ]:
Modelsaver(
    model3,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model3_SMALL')

In [ ]:
Modelsaver(
    model4,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model4_SMALL')

In [ ]:
Modelsaver(
    model5,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model5_SMALL')

In [ ]:
Modelsaver(
    model6,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model6_SMALL')

In [ ]:
Modelsaver(
    model7,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model7_SMALL')

In [ ]:
Modelsaver(
    model8,
    '/bph/puredata1/bioinfdata/user/phikie/Neural_network_results/model8_SMALL')

In [ ]:
def Loss_plotter(history, n):
    
    %matplotlib inline
    
    plt.semilogy(history.epoch, history.history['loss'],
                 color=colors[n], label='Train')
    plt.semilogy(history.epoch, history.history['val_loss'],
                color=colors[n+1], label='Val')
    
    plt.legend(loc="upper left")
    
    plt.xlabel('Epoch')
    plt.ylabel('loss')

In [ ]:
Loss_plotter(model8_history, 0)

In [ ]:
model8_history.history['AUC'][4]

In [ ]:
#Takes in a keras history and returns a datafram with the most interesting metrics of the last epoch

def Tabler(hist):
    
    table = pd.DataFrame.from_dict(hist.history)
    table2 = table.iloc[(table.shape[0]-1):table.shape[0]]
    table3 = table2[['loss', 'accuracy', 'precision', 'recall', 'AUC', 'prc', 'val_loss', 'val_accuracy', 'val_precision', 'val_recall', 'val_AUC', 'val_prc']]
    table3 = table3.reset_index(drop=True)
    return(table3)
    

In [ ]:
tabler_test = Tabler(model1_history)

In [ ]:
tabler_test

In [ ]:
model_hist_list = [model1_history,
                  model2_history,
                  model3_history,
                  model4_history,
                  model5_history,
                  model6_history,
                  model7_history,
                  model8_history]

In [ ]:
end_table = Tabler(model1_history)

for i in range(len(model_hist_list)-1):
    table_buffer = Tabler(model_hist_list[i+1])
    end_table = end_table.append(table_buffer)
end_table = end_table.reset_index(drop=True)

In [ ]:
end_table

In [ ]:
#NN_model = Sequential()
#NN_model.add(Dense(512, input_dim=(train_signals.shape[1]), activation='relu'))
#NN_model.add(Dense(256, activation='relu'))
#NN_model.add(Dense(128, activation='relu'))
#NN_model.add(Dense(64, activation='relu'))
#NN_model.add(Dense(1, activation='sigmoid'))

In [ ]:
#NN_model.fit(train_signals, train_classes, epochs=10, batch_size=32)

In [ ]:
#NN_model.save('/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Neural_networks/NN_1_B32_adam.h5')

In [ ]:
#NN_model_load = keras.models.load_model("/bph/puredata1/bioinfdata/user/phikie/Big_data_set_creation/Neural_networks/NN_1_B32_adam.h5")

In [ ]:
#val_pred = (NN_model_load.predict(val_signals) > 0.5).astype('int32')

In [ ]:
#Confus_analyzer(val_classes, val_pred)

In [ ]:
#TN, FP, FN, TP = confusion_matrix(val_classes, val_pred).ravel()

In [ ]:
%matplotlib inline

x = np.linspace(0, 69, 69)
y = val_signals[100020]

plt.plot(x, y, 'o', color='black');